# Desafio 6

Neste desafio, vamos praticar _feature engineering_, um dos processos mais importantes e trabalhosos de ML. Utilizaremos o _data set_ [Countries of the world](https://www.kaggle.com/fernandol/countries-of-the-world), que contém dados sobre os 227 países do mundo com informações sobre tamanho da população, área, imigração e setores de produção.

> Obs.: Por favor, não modifique o nome das funções de resposta.

## _Setup_ geral

In [155]:
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn as sk
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
# # Algumas configurações para o matplotlib.
# %matplotlib inline

# from IPython.core.pylabtools import figsize


# figsize(12, 8)

# sns.set()

In [156]:
countries = pd.read_csv("C:\\Users\\Luiz Guilherme\\codenation\\data-science-4\\countries.csv")

In [157]:
new_column_names = [
    "Country", "Region", "Population", "Area", "Pop_density", "Coastline_ratio",
    "Net_migration", "Infant_mortality", "GDP", "Literacy", "Phones_per_1000",
    "Arable", "Crops", "Other", "Climate", "Birthrate", "Deathrate", "Agriculture",
    "Industry", "Service"
]

countries.columns = new_column_names

countries.head(1)

,Country,Region,Population,Area,Pop_density,Coastline_ratio,Net_migration,Infant_mortality,GDP,Literacy,Phones_per_1000,Arable,Crops,Other,Climate,Birthrate,Deathrate,Agriculture,Industry,Service
0,Afghanistan,ASIA (EX. NEAR EAST),31056997,647500,"48,0","0,00","23,06","163,07",700.0,"36,0","3,2","12,13","0,22","87,65",1,"46,6","20,34","0,38","0,24","0,38"


## Observações

Esse _data set_ ainda precisa de alguns ajustes iniciais. Primeiro, note que as variáveis numéricas estão usando vírgula como separador decimal e estão codificadas como strings. Corrija isso antes de continuar: transforme essas variáveis em numéricas adequadamente.

Além disso, as variáveis `Country` e `Region` possuem espaços a mais no começo e no final da string. Você pode utilizar o método `str.strip()` para remover esses espaços.

## Inicia sua análise a partir daqui

In [158]:
z = list(countries.columns[4:8])+list(countries.columns[9:14])+list(countries.columns[15:20])
for i in range(0,len(z)):
    countries[z[i]] = [z.replace(",",".") if type(z)==str else z for z in countries[z[i]]]
    countries[z[i]] = pd.to_numeric(countries[z[i]])

## Questão 1

Quais são as regiões (variável `Region`) presentes no _data set_? Retorne uma lista com as regiões únicas do _data set_ com os espaços à frente e atrás da string removidos (mas mantenha pontuação: ponto, hífen etc) e ordenadas em ordem alfabética.

In [159]:
def q1():
    string = list(countries.Region.unique())
    return sorted([x.strip() for x in string])

## Questão 2

Discretizando a variável `Pop_density` em 10 intervalos com `KBinsDiscretizer`, seguindo o encode `ordinal` e estratégia `quantile`, quantos países se encontram acima do 90º percentil? Responda como um único escalar inteiro.

In [160]:
def q2():
    kbd = KBinsDiscretizer(n_bins=10, encode='ordinal', strategy='quantile')
    return int(np.unique(list(kbd.fit_transform(countries[['Pop_density']])),return_counts=True)[1][9])

# Questão 3

Se codificarmos as variáveis `Region` e `Climate` usando _one-hot encoding_, quantos novos atributos seriam criados? Responda como um único escalar.

In [161]:
def q3():
    return len(countries['Region'].unique())+len(countries['Climate'].unique())

## Questão 4

Aplique o seguinte _pipeline_:

1. Preencha as variáveis do tipo `int64` e `float64` com suas respectivas medianas.
2. Padronize essas variáveis.

Após aplicado o _pipeline_ descrito acima aos dados (somente nas variáveis dos tipos especificados), aplique o mesmo _pipeline_ (ou `ColumnTransformer`) ao dado abaixo. Qual o valor da variável `Arable` após o _pipeline_? Responda como um único float arredondado para três casas decimais.

In [164]:
test_country = ['Test Country', 'NEAR EAST', -0.19032480757326514,-0.3232636124824411, -0.04421734470810142, 
                -0.27528113360605316,0.13255850810281325, -0.8054845935643491, 1.0119784924248225,
                0.6189182532646624, 1.0074863283776458, 0.20239896852403538,-0.043678728558593366, 
                -0.13929748680369286, 1.3163604645710438,-0.3699637766938669, -0.6149300604558857, 
                -0.854369594993175,0.263445277972641, 0.5712416961268142]

features = []
for i in range(0,len(countries.columns)):
    if countries.dtypes[i] == (('float64') or ('int64')):
        features.append(countries.columns[i])
        
steps = [('imputer',SimpleImputer(strategy='median')),('scaler',StandardScaler())]
transformer = Pipeline(steps=steps)
preprocessor = ColumnTransformer(transformers=[('pipe',transformer, features)])
preprocessor.fit(countries)
test_country = pd.DataFrame(data=dict(zip(countries.columns,test_country)),index=[0])
test_country[features] = preprocessor.transform(test_country)
r4 = round(float(test_country['Arable']),3)

In [165]:
def q4():
    return r4

## Questão 5

Descubra o número de _outliers_ da variável `Net_migration` segundo o método do _boxplot_, ou seja, usando a lógica:

$$x \notin [Q1 - 1.5 \times \text{IQR}, Q3 + 1.5 \times \text{IQR}] \Rightarrow x \text{ é outlier}$$

que se encontram no grupo inferior e no grupo superior.

Você deveria remover da análise as observações consideradas _outliers_ segundo esse método? Responda como uma tupla de três elementos `(outliers_abaixo, outliers_acima, removeria?)` ((int, int, bool)).

In [166]:
def q5():
    Q1 = countries['Net_migration'].quantile(0.25)
    Q3 = countries['Net_migration'].quantile(0.75)
    IQR = Q3 - Q1
    r5 = (countries['Net_migration'][countries['Net_migration'] < Q1 - 1.5 * IQR].count(),
    countries['Net_migration'][countries['Net_migration'] > Q3 + 1.5 * IQR].count(),
    False)
    return r5

## Questão 6
Para as questões 6 e 7 utilize a biblioteca `fetch_20newsgroups` de datasets de test do `sklearn`

Considere carregar as seguintes categorias e o dataset `newsgroups`:

```
categories = ['sci.electronics', 'comp.graphics', 'rec.motorcycles']
newsgroup = fetch_20newsgroups(subset="train", categories=categories, shuffle=True, random_state=42)
```


Aplique `CountVectorizer` ao _data set_ `newsgroups` e descubra o número de vezes que a palavra _phone_ aparece no corpus. Responda como um único escalar.

In [13]:
categories = ['sci.electronics', 'comp.graphics', 'rec.motorcycles']
newsgroup = fetch_20newsgroups(subset="train", categories=categories, shuffle=True, random_state=42)
cv = CountVectorizer()
matrix = cv.fit_transform(newsgroup.data)
counts = pd.DataFrame(matrix.toarray(),columns=cv.get_feature_names())
r6 = int(counts['phone'].sum())

In [14]:
def q6():
    return r6

## Questão 7

Aplique `TfidfVectorizer` ao _data set_ `newsgroups` e descubra o TF-IDF da palavra _phone_. Responda como um único escalar arredondado para três casas decimais.

In [15]:
categories = ['sci.electronics', 'comp.graphics', 'rec.motorcycles']
newsgroup = fetch_20newsgroups(subset="train", categories=categories, shuffle=True, random_state=42)
tfidf = TfidfVectorizer()
matrix = tfidf.fit_transform(newsgroup.data)
idx = tfidf.vocabulary_.get('phone')
r7 = round(matrix[0:,idx].toarray().sum(),3)

In [16]:
def q7():
    return r7